In [17]:
pip install praw

In [1]:
import requests
import datetime
import pandas as pd
import numpy as np
from datetime import datetime
import praw
import re
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVC, SVR
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error, r2_score
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('vader_lexicon')


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


True

In [2]:
reddit = praw.Reddit(
    client_id="TcR41XaT0k6m_vl3tXD4Kg",
    client_secret="c4YbqCGTUGChFhxHdktDydEN4lHBNw",
    user_agent="Uni Project"
)

def convert_utc_to_date(utc_timestamp):
    return datetime.utcfromtimestamp(utc_timestamp).strftime('%d %m %Y')

def is_from_2024(utc_timestamp):
    post_date = datetime.utcfromtimestamp(utc_timestamp)
    return post_date.year == 2024

keyword = "Bitcoin"
subreddit = reddit.subreddit("cryptocurrency")

post_data = []
limit = 500
results_count = 0

for post in subreddit.new(limit=limit):
    if is_from_2024(post.created_utc):
        text = post.title
        created_date = convert_utc_to_date(post.created_utc)

        post_data.append([text, created_date])
        results_count += 1
        if results_count >= limit:
            break

df1 = pd.DataFrame(post_data, columns=["Text", "Date"])

df1['Date'] = pd.to_datetime(df1['Date'], format='%d %m %Y')

print(df1.head())




It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/latest/getting_started/multiple_instances.html#discord-bots-and-asynchronous-environments for more info.

It is strongly recommended to use Async PRAW: https://asyncpraw.readthedocs.io.
See https://praw.readthedocs.io/en/l

                                                Text       Date
0  7 things I learned during the bear market (ver... 2024-12-07
1  Bitcoin Mining Company Marathon Digital has ac... 2024-12-07
2  Czech Republic scraps capital gains tax on cry... 2024-12-07
3    Cardano and Solana, Paul Atkins and David Sacks 2024-12-07
4  Why is Algorand’s Community still so “small” d... 2024-12-07


In [3]:
def fetch_bitcoin_news(api_key):
    url = "https://newsapi.org/v2/everything"
    params = {
        "q": "bitcoin",
        "apiKey": api_key,
        "language": "en",
        "sortBy": "popularity",
        "pageSize": 10,
        "page": 1
    }

    response = requests.get(url, params=params)
    articles = response.json().get("articles", [])

    def convert_date(date_str):
        try:
            date_obj = datetime.strptime(date_str, "%Y-%m-%dT%H:%M:%SZ")
            return date_obj.strftime("%Y %m %d")
        except ValueError:
            return None

    news_data = []
    for article in articles:
        title = article['title']
        published_at = convert_date(article['publishedAt'])
        news_data.append([title, published_at])

    df = pd.DataFrame(news_data, columns=["Text", "Date"])
    df['Date'] = pd.to_datetime(df['Date'], format='%Y %m %d')
    return df

api_key = "eab43d8a7b764109aac501dce4429060"
bitcoin_news_df = fetch_bitcoin_news(api_key)

print(bitcoin_news_df)




                                                Text       Date
0  As Trump Takes a Victory Lap, the Crypto Faith... 2024-11-06
1     What Donald Trump's Win Will Mean for Big Tech 2024-11-07
2  The Crypto Industry Is Helping Trump Pick SEC ... 2024-11-26
3  Creator of Crypto Mixer ‘Bitcoin Fog’ Gets 12 ... 2024-11-09
4  Laptop That Helped Bring Down Bitcoin Thief ‘R... 2024-11-20
5  The Worst People in the World Are Making a Ton... 2024-11-06
6  Johnny Depp Impersonators Are Swindling Elderl... 2024-12-04
7  'I was scammed out of £75k by Martin Lewis dee... 2024-11-24
8  Bitcoin buyer sues council over £600m 'lost in... 2024-12-03
9  Australia's proposed social media ban for kids... 2024-11-23


In [4]:
def fetch_bitcoin_data(start_date, end_date):
    start_timestamp = int(start_date.timestamp())
    end_timestamp = int(end_date.timestamp())


    cg_url = f'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart/range'
    cg_params = {
        'vs_currency': 'usd',
        'from': start_timestamp,
        'to': end_timestamp
    }

    cg_response = requests.get(cg_url, params=cg_params)
    cg_data = cg_response.json()

    if 'prices' in cg_data:
        prices = cg_data['prices']
        df = pd.DataFrame(prices, columns=['timestamp', 'price'])
        df['date'] = pd.to_datetime(df['timestamp'], unit='ms')
        df.drop(columns=['timestamp'], inplace=True)


        volumes = cg_data.get('total_volumes', [])
        if volumes:
            df['volume'] = [v[1] for v in volumes]
        else:
            df['volume'] = np.nan


        cc_url = f'https://min-api.cryptocompare.com/data/v2/histoday'
        cc_params = {
            'fsym': 'BTC',
            'tsym': 'USD',
            'toTs': end_timestamp,
            'limit': 2000,
            'aggregate': 1,
            'api_key': 'c5ad40ff646d6c65db20d892476fefc48046b1776a7b9aae2d0d3bee0e36b158'
        }

        cc_response = requests.get(cc_url, params=cc_params)
        cc_data = cc_response.json()

        if 'Data' in cc_data:
            ohlc_data = cc_data['Data']['Data']
            ohlc_df = pd.DataFrame(ohlc_data)


            df['open'] = ohlc_df['open']
            df['high'] = ohlc_df['high']
            df['low'] = ohlc_df['low']
            df['close'] = ohlc_df['close']
        else:
            print("Error: Failed to fetch OHLC data from CryptoCompare.")

        df['date'] = df['date'].dt.strftime('%d %b %Y')

        return df
    else:
        print("No price data found.")
        return pd.DataFrame()

def calculate_indicators(df):
    # Simple Moving Averages (SMA)
    df['SMA_20'] = df['price'].rolling(window=20).mean()
    df['SMA_50'] = df['price'].rolling(window=50).mean()

    # Exponential Moving Averages (EMA)
    df['EMA_20'] = df['price'].ewm(span=20, adjust=False).mean()

    # Relative Strength Index (RSI)
    delta = df['price'].diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)

    avg_gain = gain.rolling(window=14).mean()
    avg_loss = loss.rolling(window=14).mean()

    rs = avg_gain / avg_loss
    df['RSI_14'] = 100 - (100 / (1 + rs))

    # Bollinger Bands
    df['BB_middle'] = df['price'].rolling(window=20).mean()
    df['BB_stddev'] = df['price'].rolling(window=20).std()
    df['BB_upper'] = df['BB_middle'] + (2 * df['BB_stddev'])
    df['BB_lower'] = df['BB_middle'] - (2 * df['BB_stddev'])

    # Fibonacci retracement
    highest_price = df['price'].max()
    lowest_price = df['price'].min()
    diff = highest_price - lowest_price
    df['Fibonacci_23_6'] = highest_price - 0.236 * diff
    df['Fibonacci_38_2'] = highest_price - 0.382 * diff
    df['Fibonacci_50'] = highest_price - 0.5 * diff
    df['Fibonacci_61_8'] = highest_price - 0.618 * diff

    # On-Balance Volume (OBV)
    df['OBV'] = (np.sign(df['price'].diff()) * df['price'].diff().abs()).fillna(0).cumsum()

    # Stochastic Oscillator
    df['Stochastic_K'] = 100 * (df['price'] - df['price'].rolling(window=14).min()) / (df['price'].rolling(window=14).max() - df['price'].rolling(window=14).min())
    df['Stochastic_D'] = df['Stochastic_K'].rolling(window=3).mean()

    # Aroon Indicator
    df['Aroon_Up'] = ((df['price'].rolling(window=14).apply(lambda x: np.argmax(x) + 1) / 14) * 100)
    df['Aroon_Down'] = ((df['price'].rolling(window=14).apply(lambda x: np.argmin(x) + 1) / 14) * 100)


    # Tenkan-sen (Conversion Line)
    df['Tenkan_sen'] = (df['high'].rolling(window=9).max() + df['low'].rolling(window=9).min()) / 2

    # Kijun-sen (Base Line)
    df['Kijun_sen'] = (df['high'].rolling(window=26).max() + df['low'].rolling(window=26).min()) / 2

    return df


start_date = datetime(2023, 1, 1)
end_date = datetime.utcnow()

bitcoin_data = fetch_bitcoin_data(start_date, end_date)

if not bitcoin_data.empty:
    bitcoin_data['date'] = pd.to_datetime(bitcoin_data['date'], format='%d %b %Y')
    bitcoin_data = bitcoin_data[bitcoin_data['date'] >= '2023-03-01']
    bitcoin_data_with_indicators = calculate_indicators(bitcoin_data)
    bitcoin_data_with_indicators = bitcoin_data_with_indicators.dropna()  # Remove rows with NaN values

# Display the resulting dataframe with indicators
print(bitcoin_data_with_indicators.head())



            price       date        volume     open     high      low  \
108  30365.904168 2023-04-19  1.807854e+10  8391.52  8434.63  8105.02   
109  28833.217501 2023-04-20  2.413662e+10  8246.30  8267.11  8021.30   
110  28255.578249 2023-04-21  2.169590e+10  8168.05  8232.25  8052.51   
111  27300.157129 2023-04-22  2.042033e+10  8172.48  8221.03  7822.65   
112  27861.640663 2023-04-23  1.170023e+10  7868.31  8326.01  7798.09   

       close        SMA_20        SMA_50        EMA_20  ...  Fibonacci_38_2  \
108  8246.30  29046.246194  26549.339513  29100.898112  ...     69498.41769   
109  8168.05  29085.850915  26662.892988  29075.404720  ...     69498.41769   
110  8172.48  29072.790591  26755.317884  28997.326009  ...     69498.41769   
111  7868.31  29012.851215  26832.034295  28835.690877  ...     69498.41769   
112  8215.11  28994.107812  26941.918448  28742.924190  ...     69498.41769   

     Fibonacci_50  Fibonacci_61_8          OBV  Stochastic_K  Stochastic_D  \
108  600

In [5]:
bitcoin_data_with_indicators['Date'] = pd.to_datetime(bitcoin_data_with_indicators['date'])
bitcoin_data_with_indicators.sort_values('Date', inplace=True)

projection =14
bitcoin_data_with_indicators['predict'] =bitcoin_data_with_indicators['price'].shift(-projection)
bitcoin_data_with_indicators.drop(columns=['date'], inplace=True)



<ipython-input-5-c3d34264bd1c>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_data_with_indicators['Date'] = pd.to_datetime(bitcoin_data_with_indicators['date'])
<ipython-input-5-c3d34264bd1c>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bitcoin_data_with_indicators.sort_values('Date', inplace=True)
<ipython-input-5-c3d34264bd1c>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-

In [6]:
bitcoin_data_with_indicators_clean = bitcoin_data_with_indicators.dropna()
bitcoin_data_with_indicators_clean = bitcoin_data_with_indicators.dropna(subset=['predict'])
print(bitcoin_data_with_indicators_clean.head())


            price        volume     open     high      low    close  \
108  30365.904168  1.807854e+10  8391.52  8434.63  8105.02  8246.30   
109  28833.217501  2.413662e+10  8246.30  8267.11  8021.30  8168.05   
110  28255.578249  2.169590e+10  8168.05  8232.25  8052.51  8172.48   
111  27300.157129  2.042033e+10  8172.48  8221.03  7822.65  7868.31   
112  27861.640663  1.170023e+10  7868.31  8326.01  7798.09  8215.11   

           SMA_20        SMA_50        EMA_20     RSI_14  ...  Fibonacci_61_8  \
108  29046.246194  26549.339513  29100.898112  70.165126  ...    50670.662767   
109  29085.850915  26662.892988  29075.404720  54.580668  ...    50670.662767   
110  29072.790591  26755.317884  28997.326009  51.443116  ...    50670.662767   
111  29012.851215  26832.034295  28835.690877  46.138413  ...    50670.662767   
112  28994.107812  26941.918448  28742.924190  49.390839  ...    50670.662767   

             OBV  Stochastic_K  Stochastic_D   Aroon_Up  Aroon_Down  \
108  7210.36042

In [7]:
def preprocess_text(df, column_name):

    lemmatizer = WordNetLemmatizer()

    def clean_text(text):
        text = text.lower()
        text = re.sub(r'http\S+|www\S+', '', text)
        text = re.sub(r'\d+', '', text)
        text = text.translate(str.maketrans('', '', string.punctuation))
        tokens = word_tokenize(text)
        stop_words = set(stopwords.words('english')).union(set(ENGLISH_STOP_WORDS))
        tokens = [word for word in tokens if word not in stop_words]
        tokens = [lemmatizer.lemmatize(word) for word in tokens]

        return ' '.join(tokens)


    df['preprocessed_text'] = df[column_name].apply(clean_text)


    return df[['Date', 'preprocessed_text']]
df1 = preprocess_text(df1, 'Text')
bitcoin_news_df = preprocess_text(bitcoin_news_df, 'Text')

In [8]:
sid = SentimentIntensityAnalyzer()
def get_sentiment(text):
    sentiment_score = sid.polarity_scores(text)
    return sentiment_score['compound']

df1['Sentiment_Reddit'] = df1['preprocessed_text'].apply(get_sentiment)
bitcoin_news_df['Sentiment_News'] = bitcoin_news_df['preprocessed_text'].apply(get_sentiment)

In [9]:
df_reddit = df1.groupby('Date')['Sentiment_Reddit'].mean().reset_index()
df_news = bitcoin_news_df.groupby('Date')['Sentiment_News'].mean().reset_index()


In [10]:
df_reddit['Date'] = pd.to_datetime(df_reddit['Date'], format='%d %b %Y', errors='coerce')
df_news['Date'] = pd.to_datetime(df_news['Date'], format='%d %b %Y', errors='coerce')

bitcoin_data = pd.merge(bitcoin_data_with_indicators, df_reddit, on='Date', how='left')
bitcoin_data = pd.merge(bitcoin_data, df_news, on='Date', how='left')


for column in bitcoin_data.select_dtypes(include=['number']):
    mean_value = bitcoin_data[column].mean()  # Calculate mean
    bitcoin_data[column].fillna(mean_value, inplace=True)  # Fill NaNs with mean value



<ipython-input-10-c2054d69c10e>:10: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  bitcoin_data[column].fillna(mean_value, inplace=True)  # Fill NaNs with mean value


In [12]:
bitcoin_data = bitcoin_data.sort_values(by='Date').reset_index(drop=True)

X = bitcoin_data.drop(columns=['predict', 'Date'])
y = bitcoin_data['predict']


scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [13]:
models = {
    'Random Forest': (RandomForestRegressor(random_state=42), {
        'n_estimators': [100, 200],
        'max_depth': [10, 20],
        'min_samples_split': [2, 5]
    }),
    'Support Vector Regressor': (SVR(), {
        'kernel': ['linear', 'rbf'],
        'C': [1, 10],
        'epsilon': [0.1, 0.2]
    }),
    'Linear Regression': (LinearRegression(), {
        'fit_intercept': [True, False]
    }),
    'Gradient Boosting Regressor': (GradientBoostingRegressor(random_state=42), {
        'n_estimators': [100, 200],
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5]
    })
}


evaluation_results = {}


for model_name, (model, param_grid) in models.items():

    grid_search = RandomizedSearchCV(model, param_distributions=param_grid, n_iter=5, cv=3, n_jobs=-1, random_state=42)
    grid_search.fit(X_train, y_train)


    y_pred = grid_search.best_estimator_.predict(X_test)

    mae = mean_absolute_error(y_test, y_pred)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)


    evaluation_results[model_name] = {
        'Best Params': grid_search.best_params_,
        'MAE': mae,
        'MSE': mse,
        'R2': r2
    }


for model_name, results in evaluation_results.items():
    print(f"\n{model_name} Evaluation:")
    print(f"Best Params: {results['Best Params']}")
    print(f"MAE: {results['MAE']}")
    print(f"MSE: {results['MSE']}")
    print(f"R2: {results['R2']}")



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_search.py:320: UserWarning: The total space of parameters 2 is smaller than n_iter=5. Running 2 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(



Random Forest Evaluation:
Best Params: {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': 20}
MAE: 942.6390490662536
MSE: 2435642.4398137885
R2: 0.9929717780814655

Support Vector Regressor Evaluation:
Best Params: {'kernel': 'rbf', 'epsilon': 0.1, 'C': 10}
MAE: 15793.963020461406
MSE: 332119175.2156628
R2: 0.04164616749133199

Linear Regression Evaluation:
Best Params: {'fit_intercept': False}
MAE: 6085.54864237195
MSE: 115097618.20955703
R2: 0.6678775218199251

Gradient Boosting Regressor Evaluation:
Best Params: {'n_estimators': 200, 'max_depth': 5, 'learning_rate': 0.1}
MAE: 994.7785993230921
MSE: 3576966.0232809875
R2: 0.9896784065691521
